# Chapter 7 - Strategy, Abstract Factory, and Singleton Design Patterns

Object creation patterns from the Gang of Four (GoF)

- Good for enapsulating and reusing behaviors

## Strategy Design Pattern

- Allows us to change object behaviors at runtime
- The backbone of DI
- Follows OCP

Goal

Aims to extract an algorithm (strategy) from the host class that needs it (context or consumer). This allows the consumer to decide on the strategy at runtime.



In [8]:
classDiagram
    class Consumer {
        +SomeOperation() void
    }
    class IStrategy {
        +ExecuteAlgorithm() void
    }
    class Strategy1 {
        +ExecuteAlgorithm() void
    }
    class Strategy2 {
        +ExecuteAlgorithm() void
    }
    <<Interface>> IStrategy
    IStrategy <|-- Strategy1
    IStrategy <|-- Strategy2
    Consumer o-- IStrategy


- Key is that the consumer should not know which strategy is being executed.

### Stategies

In [ ]:
public interface ISortStrategy
{
    IOrderedEnumerable<string> Sort(IEnumerable<string> input);
}

public class SortAscendingStrategy : ISortStrategy
{
 public IOrderedEnumerable<string> Sort(IEnumerable<string> input)
    => input.OrderBy(x => x);
}

public class SortDescendingStrategy : ISortStrategy
{
 public IOrderedEnumerable<string> Sort(IEnumerable<string> input)
    => input.OrderByDescending(x => x);
}

### Consumer

In [11]:
using System.Collections.Immutable;

public sealed class SortableCollection
{
    private ISortStrategy _sortStrategy;
    private ImmutableArray<string> _items;
    public IEnumerable<string> Items => _items;
    
    public SortableCollection(IEnumerable<string> items)
    {
        _items = items.ToImmutableArray();
        _sortStrategy = new SortAscendingStrategy();
    }

    public void SetSortStrategy(ISortStrategy strategy) => _sortStrategy = strategy;

    public void Sort()
    {
        _items = _sortStrategy
        .Sort(Items)
        .ToImmutableArray()
        ;
    }
}

### Usage example

In [21]:
using Microsoft.DotNet.Interactive;

SortableCollection data = new(new[] { "Lorem", "ipsum", "dolor", "sit", "amet." });

var input = await Kernel.GetInputAsync( "Sort: Enter 1 for Ascending and 2 for Descending", typeHint: "number" );
int.TryParse(input,out int sort);

ISortStrategy strategy = sort == 1
 ? new SortAscendingStrategy()
 : new SortDescendingStrategy();

data.SetSortStrategy(strategy);
data.Sort();
display( data );

Items,"[ sit, Lorem, ipsum, dolor, amet. ]"


## Abstract Factory design pattern

Use factories to create complex objects. Goal to to abstract the creation of a family of objects.

### Diagram

In [23]:
classDiagram
    class IVehicleFactory {
        +CreateCar() ICar
        +CreateBike() IBike
    }
    class HighEndVehicleFactory {
        +CreateCar() ICar
        +CreateBike() IBike
    }
    class LowEndVehicleFactory {
        +CreateCar() ICar
        +CreateBike() IBike
    }
    <<Interface>> IVehicleFactory
    IVehicleFactory <|-- HighEndVehicleFactory
    IVehicleFactory <|-- LowEndVehicleFactory


### Project: Abstract Factory

In [24]:
public interface ICar { }
public interface IBike { }
public class LowEndCar : ICar { }
public class LowEndBike : IBike { }
public class HighEndCar : ICar { }
public class HighEndBike : IBike { }

public interface IVehicleFactory
{
    ICar CreateCar();
    IBike CreateBike();
}

public class LowEndVehicleFactory : IVehicleFactory
{
    public IBike CreateBike() => new LowEndBike();
    public ICar CreateCar() => new LowEndCar();
}

public class HighEndVehicleFactory : IVehicleFactory
{
    public IBike CreateBike() => new HighEndBike();
    public ICar CreateCar() => new HighEndCar();
}

In [25]:
#r "nuget:xunit"

Installed Packages xunit, 2.9.3

In [26]:
#load "SimpleTestRunner.cs"

In [33]:
using Xunit;

public abstract class BaseAbstractFactoryTest<TConcreteFactory, TExpectedCar, TExpectedBike> where TConcreteFactory : IVehicleFactory, new()
{
    [Fact]
    public void Should_create_a_ICar_of_type_TExpectedCar()
    {
        // Arrange
        IVehicleFactory vehicleFactory = new TConcreteFactory();
        var expectedCarType = typeof(TExpectedCar);
        // Act
        ICar result = vehicleFactory.CreateCar();
        // Assert
        Assert.IsType(expectedCarType, result);
    }

    [Fact]
    public void Should_create_a_IBike_of_type_TExpectedBike()
    {
        // Arrange
        IVehicleFactory vehicleFactory = new TConcreteFactory();
        var expectedBikeType = typeof(TExpectedBike);
        // Act
        IBike result = vehicleFactory.CreateBike();
        // Assert
        Assert.IsType(expectedBikeType, result);
    }
}

public class LowEndVehicleFactoryTest : BaseAbstractFactoryTest<LowEndVehicleFactory, LowEndCar, LowEndBike>
{ 
}

public class HighEndVehicleFactoryTest : BaseAbstractFactoryTest<HighEndVehicleFactory, HighEndCar, HighEndBike>
{
}

In [34]:
SimpleTestRunner.RunTests(typeof(LowEndVehicleFactoryTest));

Running tests in LowEndVehicleFactoryTest...
Running Should_create_a_ICar_of_type_TExpectedCar...
✔ Should_create_a_ICar_of_type_TExpectedCar passed.
Running Should_create_a_IBike_of_type_TExpectedBike...
✔ Should_create_a_IBike_of_type_TExpectedBike passed.


In [35]:
SimpleTestRunner.RunTests(typeof(HighEndVehicleFactoryTest));

Running tests in HighEndVehicleFactoryTest...
Running Should_create_a_ICar_of_type_TExpectedCar...
✔ Should_create_a_ICar_of_type_TExpectedCar passed.
Running Should_create_a_IBike_of_type_TExpectedBike...
✔ Should_create_a_IBike_of_type_TExpectedBike passed.


## Singleton Design Pattern

- Application can create and reuse only 1 instance of a class
- Not everything is doable in a static class i.e. cannot implement an interface, pass as an argument, etc
- In C# we should use DI to accomplish this
- A singleton encapsulates both the object logic itself and its creational log (breaks SRP)

### Diagram

In [38]:
classDiagram
    class Singleton {
        - instance: Singleton
        + Create() Singleton
        - Singleton()
    }

In [39]:
public class MySingleton
{
    private static MySingleton? _instance;
    private MySingleton() { }

    public static MySingleton Create() //Could name GetInstance()
    {
        _instance ??= new MySingleton(); //??= null-coalescing operator
        return _instance;
    }
}

### Sidbar: `??=` null-coalescing operator

```csharp
_instance ??= new MySingleton();
```

Equals

```csharp
if (_instance == null)
{
    _instance = new MySingleton();
}
```

### Test MySingleton

In [40]:
using Xunit;

public class MySingletonTest
{
    [Fact]
    public void Create_should_always_return_the_same_instance()
    {
    var first = MySingleton.Create();
    var second = MySingleton.Create();
    Assert.Same(first, second);
    }
}

In [41]:
SimpleTestRunner.RunTests(typeof(MySingletonTest));

Running tests in MySingletonTest...
Running Create_should_always_return_the_same_instance...
✔ Create_should_always_return_the_same_instance passed.


### Use locking to make it thread safe

In [ ]:
public class MySingletonWithLock
{
    private static readonly object _myLock = new();
    private static MySingletonWithLock? _instance;
    private MySingletonWithLock() { }

    public static MySingletonWithLock Create()
    {
        lock (_myLock)
        {
            _instance ??= new MySingletonWithLock();
        }
        return _instance;
 }
}

### Simplified and thread safe

```csharp
public class MySimpleSingleton
{
    public static MySimpleSingleton Instance { get; } = new();
    private MySimpleSingleton() { }
}
```

We use DI so none of this is needed anyway.

### Advice on globals

- in general they can be dangerous
- Hard to test

### Conslusion

- The Singleton pattern violates most of the SOLID principles so should be used cautiously.
- More appropriate method in DI chapter

